In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly_express as px

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

from preprocessing import *
from feature_engineering import *


device = 'cuda' if torch.cuda.is_available() else 'cpu'

NameError: name '_C' is not defined

In [ ]:
df = pd.read_csv("E-Building_Data.csv")
df['date_time'] = pd.to_datetime(df['date_time'])

In [ ]:
fe = FeatureEngineering(df) # helper class for feature enginerring
df = fe.feature_engineering(n=False, categorical_features=["season"])

In [ ]:
latitude = 49.0069
longitude = 8.4037
start_date = datetime(2022, 6, 2)
end_date = datetime(2023, 9, 30)
wf = WeatherFetcher(latitude, longitude, start_date, end_date) # helper class to fetch weather

In [ ]:
df = fe.filter_rooms_by_prefix()
prefixes = ['e0', 'e1', 'e2', 'e3']
dataframes = [df[prefix] for prefix in prefixes]


dfe0, dfe1, dfe2, dfe3 = dataframes

In [ ]:
dataframes = [dfe0, dfe1, dfe2, dfe3]

# for i in range(len(dataframes)):
#     dataframes[i] = wf.combine_weather(dataframes[i])
# dfe0, dfe1, dfe2, dfe3 = dataframes

def one_hot_encode_room_number(df):
    return pd.get_dummies(df, columns=['room_number'], dtype="int")
encoded_dataframes = [one_hot_encode_room_number(df) for df in dataframes]

dfe0, dfe1, dfe2, dfe3 = encoded_dataframes

In [ ]:
def sliding_window_forecast(data: pd.DataFrame, label_name: str, n_in: int, n_out: int, dropna: bool = True) -> pd.DataFrame:
    """
    Transforms time series data into a supervised learning format for forecasting.

    Parameters:
    - data (pd.DataFrame): The input time series data.
    - label_name (str): The name of the target column to forecast.
    - n_in (int): Number of lag observations (input sequence length).
    - n_out (int): Number of future observations to forecast (output sequence length).
    - dropna (bool): Whether to drop rows with NaN values.

    Returns:
    - pd.DataFrame: The transformed DataFrame suitable for forecasting.
    """
    if label_name not in data.columns:
        raise ValueError(f"label_name '{label_name}' is not a column in the data")

    cols, names = list(), list()

    # Input sequence (t-n, ..., t-1, t)
    for i in range(n_in, -1, -1):
        cols.append(data.shift(i))
        names += [f"{col}(t-{i})" if i > 0 else f"{col}(t)" for col in data.columns]

    # Forecast sequence (t+1, ..., t+n)
    for i in range(1, n_out + 1):
        cols.append(data[[label_name]].shift(-i))
        names += [f"{label_name}(t+{i})"]

    # Combine all columns
    data_reframed = pd.concat(cols, axis=1)
    data_reframed.columns = names

    if dropna:
        data_reframed.dropna(inplace=True)

    return data_reframed

In [ ]:
dataframes = [dfe0, dfe1, dfe2, dfe3]
[df.set_index("date_time", inplace=True) for df in dataframes]

[None, None, None, None]

In [ ]:
dataframes = [dfe0, dfe1, dfe2, dfe3]
transformed_dataframes = [sliding_window_forecast(df, "tmp", 3, 1) for df in dataframes] # forecarst the next datapoint with the previous datapoint
dfe0, dfe1, dfe2, dfe3 = transformed_dataframes

In [ ]:
def prepare_data(df):
    input_data = df.drop(["tmp(t+1)"], axis=1).values
    targets = df[["tmp(t+1)"]].values
    T = 1  # Number of timesteps to look while predicting
    D = input_data.shape[1]  # Dimensionality of the input
    N = len(input_data) - T

    # Train size: 80% of the total data size
    train_size = int(len(input_data) * 0.80)

    # Normalization of the inputs
    scaler = StandardScaler()
    scaler.fit(input_data[:train_size + T - 1])
    input_data = scaler.transform(input_data)

    # Preparing X_train and y_train
    X_train = np.zeros((train_size, T, D))
    y_train = np.zeros((train_size, 1))

    for t in range(train_size):
        X_train[t, :, :] = input_data[t:t+T]
        y_train[t] = (targets[t+T])

    # Preparing X_test and y_test
    X_test = np.zeros((N - train_size, T, D))
    y_test = np.zeros((N - train_size, 1))

    for i in range(N - train_size):
        t = i + train_size
        X_test[i, :, :] = input_data[t:t+T]
        y_test[i] = (targets[t+T])

    # Convert to torch tensors
    X_train = torch.from_numpy(X_train.astype(np.float32))
    y_train = torch.from_numpy(y_train.astype(np.float32))
    X_test = torch.from_numpy(X_test.astype(np.float32))
    y_test = torch.from_numpy(y_test.astype(np.float32))

    return X_train, y_train, X_test, y_test

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTM, self).__init__()
        self.M = hidden_dim
        self.L = layer_dim

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=layer_dim,
            batch_first=True)
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, X):
        # Check if X is batched or unbatched
        if len(X.shape) == 2:  # Unbatched input: (sequence_length, input_dim)
            X = X.unsqueeze(0)  # Add batch dimension: (1, sequence_length, input_dim)

        batch_size = X.size(0)

        # Initialize hidden state and cell state
        h0 = torch.zeros(self.L, batch_size, self.M).to(X.device)  # (num_layers, batch_size, hidden_dim)
        c0 = torch.zeros(self.L, batch_size, self.M).to(X.device)  # (num_layers, batch_size, hidden_dim)

        # Forward pass through LSTM
        out, (hn, cn) = self.lstm(X, (h0.detach(), c0.detach()))

        # Apply dropout
        out = self.dropout(out)

        # Get output from the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

def train(model, learning_rate, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_losses = []
    test_losses = []
    
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    for epoch in range(epochs):
        model.train()
        
        epoch_train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item()
        
        # Compute average training loss for the epoch
        train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(train_loss)
        
        # Compute test loss
        model.eval()
        epoch_test_loss = 0.0
        with torch.no_grad():
            for batch_X_test, batch_y_test in test_loader:
                batch_X_test, batch_y_test = batch_X_test.to(device), batch_y_test.to(device)
                test_outputs = model(batch_X_test)
                loss = criterion(test_outputs, batch_y_test)
                epoch_test_loss += loss.item()
        
        # Compute average test loss for the epoch
        test_loss = epoch_test_loss / len(test_loader)
        test_losses.append(test_loss)
        
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return train_losses, test_losses


# Prepare data for the current floor
X_train, y_train, X_test, y_test = prepare_data(dfe0)
    
# Define the LSTM model
input_size = X_train.shape[2]  # shape of D, input data
hidden_size = 64
layer_size = 1
output_size = 1
model = LSTM(input_size, hidden_size, layer_size, output_size).to(device)

    
# Train the model for the current floor
learning_rate = 0.001
epochs = 10
batch_size = 32
train_losses, test_losses = train(model, learning_rate, X_train, y_train, X_test, y_test, epochs=epochs, batch_size=batch_size)

Epoch [1/60], Train Loss: 183.4851, Test Loss: 23.3924
Epoch [10/60], Train Loss: 6.9610, Test Loss: 1.9606
Epoch [20/60], Train Loss: 4.4282, Test Loss: 1.7011
Epoch [30/60], Train Loss: 2.6428, Test Loss: 1.4375
Epoch [40/60], Train Loss: 1.6924, Test Loss: 0.8527
Epoch [50/60], Train Loss: 1.2433, Test Loss: 0.6316
Epoch [60/60], Train Loss: 1.0609, Test Loss: 0.6891


In [29]:
# # Ensure y_test is a PyTorch tensor
# if isinstance(y_test, np.ndarray):
#     y_test = torch.tensor(y_test)

# # Ensure X_test is a PyTorch tensor
# if isinstance(X_test, np.ndarray):
#     X_test = torch.tensor(X_test)

# model.eval()
# with torch.no_grad():
#     if torch.cuda.is_available():
#         X_test = X_test.cuda()
#         y_test = y_test.cuda()  # Ensure y_test is also moved to GPU if it's not already
#     y_pred = model(X_test)

# # Move predictions and true values to CPU if necessary
# if torch.cuda.is_available():
#     y_pred = y_pred.cpu()
#     y_test = y_test.cpu()

# # Convert tensors to numpy arrays
# y_pred = y_pred.numpy()
# y_test = y_test.numpy()

# Create a DataFrame with true values and predicted values
df = pd.DataFrame({
    'Index': range(len(y_test)),
    'True Values': y_test.flatten(),
    'Predicted Values': test_outputs.flatten()
})

# Melt the DataFrame for Plotly Express
df_melted = df.melt(id_vars=['Index'], value_vars=['True Values', 'Predicted Values'],
                    var_name='Type', value_name='Value')

# Plot using Plotly Express
fig = px.line(df_melted, x='Index', y='Value', color='Type', title='True vs Predicted Values')
fig.show()

In [164]:
import numpy as np
from sklearn.metrics import mean_squared_error

def feature_ablation_importance(model, data, target):
    model.eval()
    device = next(model.parameters()).device  # Get the device of model parameters
    
    data = data.to(device)
    target = target.to(device)
    
    with torch.no_grad():
        baseline_error = mean_squared_error(target.cpu().numpy(), model(data).cpu().numpy())
        importances = []

        for col in range(data.shape[2]):  # Assumes data shape is (batch_size, sequence_length, num_features)
            ablated_data = data.clone()
            ablated_data[:, :, col] = 0  # Zero out the feature column
            ablated_error = mean_squared_error(target.cpu().numpy(), model(ablated_data).cpu().numpy())
            importances.append(baseline_error - ablated_error)

    return np.array(importances)

# Ensure X_test and y_test are PyTorch tensors and moved to the correct device
if isinstance(X_test, np.ndarray):
    X_test = torch.tensor(X_test, dtype=torch.float32)
if isinstance(y_test, np.ndarray):
    y_test = torch.tensor(y_test, dtype=torch.float32)

if torch.cuda.is_available():
    X_test = X_test.cuda()
    y_test = y_test.cuda()

# Usage
importances = feature_ablation_importance(model, X_test, y_test)
print("Feature Importances:", importances)

Feature Importances: [-3.52674675e+00  3.24152112e-02  1.60783529e-03  1.73604488e-02
  1.50519609e-03  4.19380069e-02  4.50968146e-02  6.65101409e-02
  3.72347236e-02 -1.22583330e-01  1.49102211e-02 -1.42218471e-02
 -1.52993083e-01 -3.29774320e-01  3.42492461e-02  3.33873034e-02
  3.94415855e-03  2.03746557e-03  2.20714808e-02 -1.83874369e-03
  7.62093067e-03  3.81373763e-02  1.19203925e-02 -1.14050508e-02
  1.20853186e-02  1.76165104e-02  2.69243717e-02 -5.06103039e-04
  1.68727636e-02 -3.06618357e+00  2.91125178e-02 -5.63263893e-04
  1.65887475e-02  4.19855118e-04  5.10773659e-02  4.49062586e-02
  6.59358501e-02  5.15124798e-02 -1.16910517e-01  1.84397697e-02
 -5.34573197e-02 -1.52056456e-01 -2.50993133e-01  6.69086576e-02
  4.30728197e-02  2.32166052e-03  2.09212303e-05  2.74676681e-02
  6.45643473e-03  8.43060017e-03  4.10306454e-02  1.93022490e-02
  4.30816412e-03  1.80150270e-02  2.45679021e-02  2.90974379e-02
  2.53783464e-02  1.76213384e-02]


In [173]:
features = dfe0.drop(["tmp(t+1)"], axis=1)
feature_names = features.columns  # This assumes dfe0 has columns with feature names

# Create a DataFrame with real feature names and importance values
df_feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

In [176]:
df_feature_importance

,Feature,Importance
0,tmp(t-1),-3.526747
1,hum(t-1),0.032415
2,tmp_diff(t-1),0.001608
3,hour_sin(t-1),0.017360
4,hour_cos(t-1),0.001505
5,day_of_week_sin(t-1),0.041938
6,day_of_week_cos(t-1),0.045097
7,month_sin(t-1),0.066510
8,month_cos(t-1),0.037235
9,season_autumn(t-1),-0.122583


In [172]:
features = [f'Feature {i+1}' for i in range(len(importances))]
df_importance = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# Create a horizontal bar plot using Plotly Express
fig = px.bar(df_feature_importance, x='Importance', y='Feature', orientation='h', title='Feature Importance Scores')

# Show the plot
fig.show()

In [174]:
# Define window size and label name
n_in = 1  # Number of past time steps to consider for prediction
n_out = 1  # Number of future time steps to predict
label_name = "tmp(t)"  # Target variable name


# Extract the last window for prediction (excluding the target variable)
last_window = dfe0.iloc[-1, :-1].values
last_window = last_window.reshape(1, -1)

# Extract the label (target variable) for prediction
next_value_label = dfe0.iloc[-1][label_name]

# Convert to PyTorch tensor
last_window_tensor = torch.tensor(last_window, dtype=torch.float32)

In [175]:
model.eval()
with torch.no_grad():
    if torch.cuda.is_available():
        last_window_tensor = last_window_tensor.cuda()
    next_value_pred = model(last_window_tensor)

# Move prediction to CPU if necessary
if torch.cuda.is_available():
    next_value_pred = next_value_pred.cpu()

# Convert tensor to numpy array
next_value_pred = next_value_pred.numpy()

# Print the predicted next value
print(f'Predicted next value: {next_value_pred[0][0]}, True Label: {next_value_label}')

Predicted next value: 19.77297019958496, True Label: 24.37


In [25]:
import numpy as np
import torch

def predict_next_values(models, dataframes, label_names):
    predictions = []

    # Iterate over each model, dataframe, and label_name
    for model, data, label_name in zip(models, dataframes, label_names):
        # Extract the last window for prediction (excluding the target variable)
        last_window = data.iloc[-1, :-1].values
        last_window = last_window.reshape(1, -1)

        # Extract the label (target variable) for prediction
        next_value_label = data.iloc[-1][label_name]

        # Convert to PyTorch tensor
        last_window_tensor = torch.tensor(last_window, dtype=torch.float32)

        # Move tensor to GPU if available
        if torch.cuda.is_available():
            last_window_tensor = last_window_tensor.cuda()

        # Set model to evaluation mode
        model.eval()

        # Perform prediction
        with torch.no_grad():
            next_value_pred = model(last_window_tensor)

        # Move prediction to CPU if necessary
        if torch.cuda.is_available():
            next_value_pred = next_value_pred.cpu()

        # Convert tensor to numpy array and get the predicted value
        next_value_pred = next_value_pred.numpy()[0][0]

        # Store the prediction along with the true label
        predictions.append((next_value_pred, next_value_label))

        # Print the predicted next value and true label
        print(f'For dataframe with label {label_name}: Predicted next value: {next_value_pred}, True Label: {next_value_label}')

    return predictions


In [26]:
label_name1 = "tmp1(t)"  # Target variable name for dataframe 1
label_name2 = "tmp(t)"  # Target variable name for dataframe 2
label_name3 = "tmp(t)"  # Target variable name for dataframe 3
label_name4 = "tmp(t)"  # Target variable name for dataframe 4

models = [model1, model2, model3, model4]
dataframes = [dfe1, dfe2, dfe3, dfe4]
label_names = [label_name1, label_name2, label_name3, label_name4]

# Call the method to predict the next values for each dataframe
predictions = predict_next_values(models, dataframes, label_names)

NameError: name 'model1' is not defined